In [1]:
import pandas as pd
import os

In [2]:
clones_reseda = pd.read_csv("AMC-TC-x001-HL-clones.csv")
clones_reseda.head()

,Unnamed: 0,CellID,CellType_x,cdr3pep_heavy,CellType_y,cdr3pep_light,freq
0,2964,GGATTACCATCTACGA-1,IGH_HUMAN,CARWDYDFWSGYYTGRASSGANFDYW,IGK_HUMAN,CLQHNSYPRTF,32
1,4239,TGGCGCAAGTAGCCGA-1,IGH_HUMAN,CARGGGDSSGFSPGRIDSW,IGK_HUMAN,CQQYNSYPYTF,32
2,4238,TGGCGCAAGTAGCCGA-1,IGH_HUMAN,CARGGGDSSGFSPGRIDSW,IGK_HUMAN,CQQYNNWLTWTF,32
3,4237,TGGCGCAAGTAGCCGA-1,IGH_HUMAN,CARGGGDSSGFSPGRIDSW,IGK_HUMAN,CQQRSNWPPFTF,32
4,4399,TTCCCAGGTAAACCTC-1,IGH_HUMAN,CASKDLSRGFDYW,IGK_HUMAN,CQQRSNWPPRLTF,28


In [6]:
clones_cellranger = pd.read_csv("../cellranger-output/clonotypes/AMC-TC-x001-clonotypes.csv")
clones_cellranger.head()

,clonotype_id,frequency,proportion,cdr3s_aa,cdr3s_nt
0,clonotype1,14,0.003648,IGH:CAREARSAYYDYVWGSYRYGWFDPW;IGK:CMQSIQLPITF,IGH:TGTGCGAGAGAGGCTCGGAGTGCGTATTATGATTACGTTTGG...
1,clonotype2,12,0.003127,IGK:CQQYNNWPPLTF,IGK:TGTCAGCAGTATAATAACTGGCCTCCGCTCACTTTC
2,clonotype3,10,0.002606,IGH:CATDMGLGYCSSTSCYGGWFDPW;IGK:CQQRSNWPPTF,IGH:TGTGCGACCGATATGGGGCTAGGATATTGTAGTAGTACCAGC...
3,clonotype4,9,0.002345,IGK:CLQDYLYPWTF,IGK:TGTCTACAAGATTACCTTTATCCGTGGACGTTC
4,clonotype6,8,0.002084,IGH:CAKDAGGAGMVNGFDYW;IGL:CQSADSSGTYPVF,IGH:TGTGCGAAAGATGCGGGGGGGGCAGGTATGGTTAATGGGTTT...


In [8]:
def cdr3ToSeperateColumns(cdr3_aa):
    chains = cdr3_aa.split(";")
    celltype_x = None
    cdr3pep_heavy = None
    celltype_y = None
    cdr3pep_light = None
    if len(chains) > 2:
        print("More than two chains were annotated", cdr3_aa)
    for chain in chains:
        celltype, cdr3pep = chain.split(":")
        if celltype == "IGH":
            celltype_x = "IGH_HUMAN"
            cdr3pep_heavy = cdr3pep
        elif celltype == "IGK":
            celltype_y = "IGK_HUMAN"
            cdr3pep_light = cdr3pep
        elif celltype == "IGL":
            celltype_y = "IGL_HUMAN"
            cdr3pep_light = cdr3pep
        else:
            print("There should be nothing else than IGH, IGK or IGL", cdr3_aa)
    return(celltype_x, cdr3pep_heavy, celltype_y, cdr3pep_light)

print(cdr3ToSeperateColumns("IGH:CAREARSAYYDYVWGSYRYGWFDPW;IGK:CMQSIQLPITF"))
print(cdr3ToSeperateColumns("IGK:CLQDYLYPWTF"))  

('IGH_HUMAN', 'CAREARSAYYDYVWGSYRYGWFDPW', 'IGK_HUMAN', 'CMQSIQLPITF')
(None, None, 'IGK_HUMAN', 'CLQDYLYPWTF')


In [9]:
celltype_x_list = list()
cdr3pep_heavy_list = list()
celltype_y_list = list()
cdr3pep_light_list = list()
for cdr3_aa in clones_cellranger["cdr3s_aa"]:
    (celltype_x, cdr3pep_heavy, celltype_y, cdr3pep_light) = cdr3ToSeperateColumns(cdr3_aa)
    celltype_x_list.append(celltype_x)
    cdr3pep_heavy_list.append(cdr3pep_heavy)
    celltype_y_list.append(celltype_y)
    cdr3pep_light_list.append(cdr3pep_light)
clones_cellranger["CellType_x"] = celltype_x_list
clones_cellranger["cdr3pep_heavy"] = cdr3pep_heavy_list
clones_cellranger["CellType_y"] = celltype_y_list
clones_cellranger["cdr3pep_light"] = cdr3pep_light_list
clones_cellranger.head()

More than two chains were annotated IGH:CAITQYQLLGSYYYYYGMDVW;IGH:CADSFDYW;IGK:CQQRSNWPPYTF;IGK:CQQYGSSPFTWTF
More than two chains were annotated IGH:CARSAFYCSSTSCYLDAFDIW;IGK:CQQYGSSPLTF;IGK:CLQHNSYPHTF
More than two chains were annotated IGH:CAREGTIYDYVWGSYRFNWFDPW;IGK:CQQYGSSYTF;IGL:CCSYAGSRVF
More than two chains were annotated IGH:CARVQMYCSSTSCYVGNAFDIW;IGL:CGTWDSSLSVWVF;IGL:CMIWPSNAYVVF
More than two chains were annotated IGH:CARARLGHYYYGMDVW;IGH:CAVMGYGDYGYW;IGK:CQQYNSYVFTF;IGK:CQQRSNWPRSTF
More than two chains were annotated IGH:CARRDDDYSNYLLSSYGMDVW;IGH:CARDGAGGGYW;IGK:CQQYNNWPPVYTF
More than two chains were annotated IGH:CAGGGITMVRGVYYFDYW;IGH:CAKDTGFYDISGEFDYW;IGL:CQAWDSSTGVF
More than two chains were annotated IGH:CARDSKELTGDYYYYYMDVW;IGH:CARVRDYINYFDYW;IGK:CQQRSNWPPVITF;IGK:CQQYGSSPLTF
More than two chains were annotated IGH:CARDGSVEATGDLGYW;IGK:CQQYGSSIYTF;IGK:CQQRSNWPPEITF
More than two chains were annotated IGH:CAKDFFPSSYDYVWGSYRPYGAFDIW;IGK:CQQYYSTPRTF;IGK:CQQYGSSPWTF


,clonotype_id,frequency,proportion,cdr3s_aa,cdr3s_nt,CellType_x,cdr3pep_heavy,CellType_y,cdr3pep_light
0,clonotype1,14,0.003648,IGH:CAREARSAYYDYVWGSYRYGWFDPW;IGK:CMQSIQLPITF,IGH:TGTGCGAGAGAGGCTCGGAGTGCGTATTATGATTACGTTTGG...,IGH_HUMAN,CAREARSAYYDYVWGSYRYGWFDPW,IGK_HUMAN,CMQSIQLPITF
1,clonotype2,12,0.003127,IGK:CQQYNNWPPLTF,IGK:TGTCAGCAGTATAATAACTGGCCTCCGCTCACTTTC,None,None,IGK_HUMAN,CQQYNNWPPLTF
2,clonotype3,10,0.002606,IGH:CATDMGLGYCSSTSCYGGWFDPW;IGK:CQQRSNWPPTF,IGH:TGTGCGACCGATATGGGGCTAGGATATTGTAGTAGTACCAGC...,IGH_HUMAN,CATDMGLGYCSSTSCYGGWFDPW,IGK_HUMAN,CQQRSNWPPTF
3,clonotype4,9,0.002345,IGK:CLQDYLYPWTF,IGK:TGTCTACAAGATTACCTTTATCCGTGGACGTTC,None,None,IGK_HUMAN,CLQDYLYPWTF
4,clonotype6,8,0.002084,IGH:CAKDAGGAGMVNGFDYW;IGL:CQSADSSGTYPVF,IGH:TGTGCGAAAGATGCGGGGGGGGCAGGTATGGTTAATGGGTTT...,IGH_HUMAN,CAKDAGGAGMVNGFDYW,IGL_HUMAN,CQSADSSGTYPVF


In [10]:
len(clones_cellranger)

3292

In [11]:
len(clones_reseda)

4582

In [12]:
clones_reseda["CellID"].nunique()

3241